# Minimising Portfolio Risk - Multiple Assets - Applied

### The risk of a $k$ asset portfolio is calculated as...
$$\sigma_p^2 = var(\mathbf{\Omega'R})$$  
$$\sigma_p^2 = \mathbf{\Omega'\sum\Omega}$$  
$$\sigma_p = \sqrt{\sigma_p^2}$$

Where:  
$\sigma_p^2 = $ The variance of the portfolio  
$\sigma_p = $ The standard deviation (risk) of the portfolio  
$\mathbf{\Omega} = $ Vector of 'weights'  
$\mathbf{\sum} = $ Variance Covariance (VCV) Matrix  
$\mathbf{R} =$ Vector of Returns

In [10]:
# Import package dependencies
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [11]:
df = pd.read_csv(r"C:\Users\rokhs\OneDrive\Courses\Investment Analysis & Portfolio Management with Python\39\minimising_portfolio_risk\data\15stocks_price.csv")  # stock price data

# Convert dates to timestamps and set date column as the index
df['date_gsheets'] = pd.to_datetime(df['date_gsheets'])
df.set_index('date_gsheets', inplace=True)

In [12]:
df.head()

,AAPL,KO,NFLX,BRK.B,DIS,IBM,VZ,WMT,GE,TSLA,MA,AMZN,MSFT,UN,V
date_gsheets,,,,,,,,,,,,,,,
2012-01-03 16:00:00,58.75,35.07,10.32,77.68,38.31,186.30,39.73,60.33,18.36,28.08,36.84,179.03,26.77,34.92,25.75
2012-01-04 16:00:00,59.06,34.85,11.49,76.80,38.85,185.54,39.21,59.71,18.56,27.71,35.63,177.51,27.40,34.45,25.29
2012-01-05 16:00:00,59.72,34.69,11.33,76.93,39.50,184.66,38.94,59.42,18.55,27.12,35.24,177.61,27.68,34.49,25.48
2012-01-06 16:00:00,60.34,34.47,12.33,76.39,39.91,182.54,38.33,59.00,18.65,26.91,34.29,182.61,28.11,33.27,25.18
2012-01-09 16:00:00,60.25,34.47,14.03,76.29,39.75,181.59,38.37,59.18,18.86,27.25,34.58,178.56,27.74,33.72,24.98


## Creat the objective function

In [13]:
def getPortRisk(weights):
    
    '''Returns the annualised standard deviation of a k asset portfolio.'''

    returns_df = df.pct_change(1).dropna()  # estimate returns for each asset
    num_stocks = len(returns_df.columns)  # number of stocks based on number of columns (excluding index col)
                                          # this is a local variable
        
    vcv = returns_df.cov()  # being the variance covariance matrix
    
    var_p = np.dot(np.transpose(weights), np.dot(vcv, weights))  # variance of the multi-asset portfolio
    sd_p = np.sqrt(var_p)  # standard deviation of the multi-asset portfolio
    sd_p_annual = sd_p * np.sqrt(250)  # annualised standard deviation of the multi-asset portfolio
    
    return sd_p_annual

In [26]:
num_stocks = len(df.columns)  # being the number of stocks (this is a 'global' variable)
init_weights = [1 / num_stocks] * num_stocks  # initialise weights (x0)

In [27]:
init_weights

[0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667]

In [28]:
# Constraint that weights in any asset j must be between 0 and 1 inclusive
bounds = tuple((0, 1) for i in range(num_stocks))

In [29]:
bounds

((0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1))

In [30]:
# Constraint that the sum of the weights of all assets must equate to 1
cons = ({'type' : 'eq', 'fun' : lambda x : np.sum(x) - 1})

#### SciPy minimize Method within SciPy's 'optimize'

In [17]:
results = minimize(fun=getPortRisk, x0=init_weights, bounds=bounds, constraints=cons)

In [18]:
results

     fun: 0.10593479369289101
     jac: array([0.10597393, 0.10585235, 0.10600163, 0.1058007 , 0.10597585,
       0.10618784, 0.10613558, 0.10582459, 0.10587699, 0.11916338,
       0.11288869, 0.10579355, 0.10934387, 0.1058938 , 0.10634915])
 message: 'Optimization terminated successfully.'
    nfev: 187
     nit: 11
    njev: 11
  status: 0
 success: True
       x: array([4.79798172e-02, 2.80742509e-01, 7.53106161e-03, 1.29294768e-01,
       3.54257579e-02, 8.40457981e-02, 1.62830959e-01, 1.72868276e-01,
       2.00872901e-02, 1.62630326e-19, 8.67361738e-19, 3.55671929e-03,
       1.30104261e-18, 4.31645300e-02, 1.24725145e-02])

In [19]:
# Check total risk of the equal weighted portfolio
getPortRisk(init_weights)

0.13290599867193698

In [20]:
# Explore optimised weights
optimised_weights = pd.DataFrame(results['x'])
optimised_weights.index = df.columns
optimised_weights.rename(columns={optimised_weights.columns[0] : 'weights'}, inplace=True)

In [21]:
optimised_weights

,weights
AAPL,4.797982e-02
KO,2.807425e-01
NFLX,7.531062e-03
BRK.B,1.292948e-01
DIS,3.542576e-02
IBM,8.404580e-02
VZ,1.628310e-01
WMT,1.728683e-01
GE,2.008729e-02
TSLA,1.626303e-19


In [22]:
# Clean format of the weights so it's more readable
optimised_weights['weights_rounded'] = optimised_weights['weights'].apply(lambda x : round(x, 3))     # creat a new column

In [23]:
optimised_weights

,weights,weights_rounded
AAPL,4.797982e-02,0.048
KO,2.807425e-01,0.281
NFLX,7.531062e-03,0.008
BRK.B,1.292948e-01,0.129
DIS,3.542576e-02,0.035
IBM,8.404580e-02,0.084
VZ,1.628310e-01,0.163
WMT,1.728683e-01,0.173
GE,2.008729e-02,0.020
TSLA,1.626303e-19,0.000


In [24]:
# Notice how 7 of the 15 stocks make up 92.1% of the portfolio allocation!
optimised_weights['weights_rounded'].sort_values(ascending=False).cumsum()

KO       0.281
WMT      0.454
VZ       0.617
BRK.B    0.746
IBM      0.830
AAPL     0.878
UN       0.921
DIS      0.956
GE       0.976
V        0.988
NFLX     0.996
AMZN     1.000
TSLA     1.000
MA       1.000
MSFT     1.000
Name: weights_rounded, dtype: float64